In [1]:
import os 
from dotenv import load_dotenv
os.chdir("..")

In [3]:
import pandas as pd
import sqlite3

db_name = "lease_real_estate.db"
def load_csv_to_db(csv_file_path, table_name):
    """
    Loads data from a CSV file into an SQLite database.

    Parameters:
    csv_file_path (str): Path to the CSV file.
    table_name (str): Name of the table where data will be dumped.

    Returns:
    None
    """
    # Connect to the SQLite database (creates the file if it doesn't exist)
    conn = sqlite3.connect(db_name)  #updated with new data
    
    try:
        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(csv_file_path)
        

        # Dump the DataFrame into the SQLite table
        df.to_sql(table_name, conn, if_exists='replace', index=False)

        print(f"Data successfully loaded into table '{table_name}' in '{db_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        conn.close()

# Example usage
# load_csv_to_db('data.csv', 'properties')

In [4]:
for file in os.listdir("updated_data"):
    table_name = file.split(".")[0].lower()
    load_csv_to_db(f"updated_data/{file}", table_name)
    print(f"Data loaded for {table_name}")
    

Data successfully loaded into table 'lease_details' in 'lease_real_estate.db'.
Data loaded for lease_details
Data successfully loaded into table 'rent_status' in 'lease_real_estate.db'.
Data loaded for rent_status


In [5]:

load_csv_to_db(rent_data, "rent_table")
print("Rent data uploaded")
load_csv_to_db(lease_data,"lease_table")
print("lease data uploaded")

Data successfully loaded into table 'rent_table' in 'real_estate.db'.
Rent data uploaded
Data successfully loaded into table 'lease_table' in 'real_estate.db'.
lease data uploaded


In [27]:
pd.read_csv(rent_data).columns

Index(['tenant_name', 'apartment_number', 'lease_start_date', 'lease_end_date',
       'payment_date', 'payment_status', 'delay_days', 'tenant_email',
       'zip_code'],
      dtype='object')

In [26]:
pd.read_csv(lease_data).columns

Index(['tenant_name', 'apartment_number', 'email', 'tenant_phone', 'tenant_id',
       'rental_amount', 'payment_date', 'due_date', 'status', 'city',
       'apartment_building', 'lease_start_date', 'lease_end_date',
       'lease_terms_conditions', 'renewal_terms_conditions', 'owner_name',
       'owner_email', 'owner_contact_number'],
      dtype='object')

In [7]:
conn = sqlite3.connect(db_name)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)
# conn.execute("SELECT * FROM ").fetchall()

[('lease_details',), ('rent_status',)]


In [5]:
import sqlite3
from typing import Optional, Dict

def get_tenant_rent_status(tenant_name: str, apartment_number: str) -> Optional[Dict]:
    """
    Fetch current rent status for a tenant based on name and apartment number.
    
    Args:
        tenant_name (str): Name of the tenant
        apartment_number (str): Apartment number
    
    Returns:
        Optional[Dict]: Dictionary containing tenant details and rent status if found,
                       None if tenant not found
    """
    try:
        # Establish database connection
        conn = sqlite3.connect(db_name)  # Replace with your database name
        cursor = conn.cursor()
        
        # SQL query to join lease_details and rent_status tables
        query = """
        SELECT 
            l.Tenant_Name,
            l.Apartment_Number,
            l.Current_Rent,
            l.Lease_Status,
            l.Lease_End_Date,
            r.Status as Rent_Status
        FROM lease_details l
        LEFT JOIN rent_status r ON l.Tenant_ID = r.Tenant_ID
        WHERE LOWER(l.Tenant_Name) = LOWER(?) 
        AND l.Apartment_Number = ?
        """
        
        # Execute query with parameters
        cursor.execute(query, (tenant_name, apartment_number))
        result = cursor.fetchone()
        
        if result:
            return {
                'tenant_name': result[0],
                'apartment_number': result[1],
                'current_rent': result[2],
                'lease_status': result[3],
                'lease_end_date': result[4],
                'rent_status': result[5]
            }
        return None
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        raise
        
    finally:
        if conn:
            conn.close()

In [6]:
get_tenant_rent_status(tenant_name="Ishita Agarwal", apartment_number="10A")

{'tenant_name': 'Ishita Agarwal',
 'apartment_number': '10A',
 'current_rent': 1500,
 'lease_status': 'Pending',
 'lease_end_date': '31-01-2025',
 'rent_status': 'Pending'}